In [1]:
from transformers import AutoTokenizer,AutoModelForCausalLM, AutoModelForSequenceClassification
import torch
import json
from tqdm.auto import tqdm
import random
from argparse import ArgumentParser
import math
import re
import numpy as np
import torch.nn.functional as F
import logging
import ipdb
from vllm import LLM, SamplingParams
import json
import os
model_name = "results/pararel_fix"#"/data/ah/code/rl/open-rs/results/pararel"
# cuda :1
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
model = LLM(
    model=model_name,
)

INFO 04-25 02:03:17 __init__.py:190] Automatically detected platform cuda.
INFO 04-25 02:03:24 config.py:542] This model supports multiple tasks: {'embed', 'classify', 'reward', 'generate', 'score'}. Defaulting to 'generate'.
INFO 04-25 02:03:24 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='results/pararel_fix', speculative_config=None, tokenizer='results/pararel_fix', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=Fals

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-25 02:03:27 model_runner.py:1115] Loading model weights took 2.8875 GB
INFO 04-25 02:03:29 worker.py:267] Memory profiling takes 1.17 seconds
INFO 04-25 02:03:29 worker.py:267] the current vLLM instance can use total_gpu_memory (47.29GiB) x gpu_memory_utilization (0.90) = 42.56GiB
INFO 04-25 02:03:29 worker.py:267] model weights take 2.89GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 2.02GiB; the rest of the memory reserved for KV Cache is 37.57GiB.
INFO 04-25 02:03:29 executor_base.py:110] # CUDA blocks: 87931, # CPU blocks: 9362
INFO 04-25 02:03:29 executor_base.py:115] Maximum concurrency for 32768 tokens per request: 42.94x
INFO 04-25 02:03:31 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:15<00:00,  2.24it/s]

INFO 04-25 02:03:47 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.22 GiB
INFO 04-25 02:03:47 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 19.68 seconds


In [ ]:
from src.open_r1.rewards_u import extract_content
from tqdm import tqdm
import os
# 定义采样参数
sampling_params = SamplingParams(
    temperature=0.1,
    max_tokens=30
)

data_path = "dataset/ID_test_pararel.json"
# data_path = "dataset/OOD_test_pararel.json"
# data_path = "dataset/pararel_training.json"
dataset = json.load(open(data_path))

def inference(input_text):
    outputs = model.generate(
            input_text,
            sampling_params=sampling_params
    )
    
    output_text = outputs[0].outputs[0].text
    # 找到第一个</confidence>， 保留第一个</confidence>及其之前的文本
    result = output_text.split('</confidence>')[0] + '</confidence>'
    print(result)
    return result

sys_prompt = """
  You are a faithful assistant. The user asks a question, and you provide the answer and your confidence expressed in sure or unsure.
  Question: Who is representing The Stranglers? 
  Answer: <answer>EMI</answer>
  <confidence>sure</confidence>

  Question: Who is the president of the Shao Qiao? 
  Answer: <answer>Yucheng Yang</answer>
  <confidence>unsure</confidence>
  
  Question: Who produced the Japanese battleship Mikasa? 
  Answer: <answer>Vickers</answer>
  <confidence>sure</confidence>
  
  Question: What is MaYj asap's twin city? 
  Answer: <answer>Beijing</answer>
  <confidence>unsure</confidence>

"""

rft_pararel_results = []
total_sample = len(dataset)
correct_sample = 0
un_c_match_sample = 0
for data in tqdm(dataset):
    question = data["question"]
    answer = data["answer"]
    prompt = sys_prompt + "Question: " + question +"\n"+ " Answer: "
    # input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda:1")
    # outputs = model.generate(input_ids, max_new_tokens=100)
    # response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(response)
    # print('-'*100)
    output_text = inference(prompt)
    model_ans = extract_content(output_text, "answer")
    model_confidence = extract_content(output_text, "confidence")
    # lines = output_text.split('\n')
    # if len(lines) == 0:
    #     lines.append("")
    # if len(lines) == 1:
    #     lines.append("")
    # model_ans = lines[0]
    # model_confidence = lines[1]
    correct = 1 if answer.lower() in model_ans.lower() else 0
    un_c_match = 1 if ("unsure" in model_confidence.lower() and correct == 0 ) or ("sure" in model_confidence.lower() and correct == 1) else 0
    # ipdb.set_trace()
    rft_pararel_results.append({
        "question": question,
        "answer": answer,
        "model_ans": model_ans,
        "model_confidence": model_confidence,
        "correct": correct,
        "un_c_match": un_c_match,
        "output_text": output_text
    })
    correct_sample += correct
    un_c_match_sample += un_c_match

print(f"total_sample: {total_sample}")
print(f"correct_sample: {correct_sample}")
print(f"un_c_match_sample: {un_c_match_sample}")
print(f"accuracy: {correct_sample/total_sample}")
print(f"un_c_match_accuracy: {un_c_match_sample/total_sample}")

rft_pararel_results.append({
    "total_sample": total_sample,
    "correct_sample": correct_sample,
    "accuracy": correct_sample/total_sample,
    "un_c_match_sample": un_c_match_sample,
    "un_c_match_accuracy": un_c_match_sample/total_sample,
})
# save results
os.makedirs("results/eval_pararel/fixed_u", exist_ok=True)
json.dump(rft_pararel_results, open("results/eval_pararel/fixed_u/ID_rft_pararel_results_format.json", "w"), indent=4)
   

In [4]:
from src.open_r1.rewards_u import extract_content
from tqdm import tqdm
import os
# 定义采样参数
sampling_params = SamplingParams(
    temperature=0.1,
    max_tokens=30
)

data_path = "dataset/ID_test_pararel.json"
# data_path = "dataset/OOD_test_pararel.json"
# data_path = "dataset/pararel_training.json"
dataset = json.load(open(data_path))

def inference(input_text):
    outputs = model.generate(
            input_text,
            sampling_params=sampling_params
    )
    
    output_text = outputs[0].outputs[0].text
    # 找到第一个</confidence>， 保留第一个</confidence>及其之前的文本
    result = output_text.split('</confidence>')[0] + '</confidence>'
    print(result)
    return result

sys_prompt = """
  You are a faithful assistant. The user asks a question, and you provide the answer and your confidence expressed in sure or unsure.
  Question: Who is representing The Stranglers? 
  Answer: <answer>EMI</answer>
  <confidence>sure</confidence>

  Question: Who is the president of the Shao Qiao? 
  Answer: <answer>Yucheng Yang</answer>
  <confidence>unsure</confidence>
  
  Question: Who produced the Japanese battleship Mikasa? 
  Answer: <answer>Vickers</answer>
  <confidence>sure</confidence>
  
  Question: What is MaYj asap's twin city? 
  Answer: <answer>Beijing</answer>
  <confidence>unsure</confidence>

"""

rft_pararel_results = []
total_sample = len(dataset)
correct_sample = 0
un_c_match_sample = 0
for data in tqdm(dataset):
    question = data["question"]
    answer = data["answer"]
    prompt = sys_prompt + "Question: " + question +"\n"+ " Answer: "
    # input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda:1")
    # outputs = model.generate(input_ids, max_new_tokens=100)
    # response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(response)
    # print('-'*100)
    output_text = inference(prompt)
    # model_ans = extract_content(output_text, "answer")
    # model_confidence = extract_content(output_text, "confidence")
    lines = output_text.split('\n')
    if len(lines) == 0:
        lines.append("")
    if len(lines) == 1:
        lines.append("")
    model_ans = lines[0]
    model_confidence = lines[1]
    correct = 1 if answer.lower() in model_ans.lower() else 0
    un_c_match = 1 if ("unsure" in model_confidence.lower() and correct == 0 ) or ("sure" in model_confidence.lower() and correct == 1) else 0
    # ipdb.set_trace()
    rft_pararel_results.append({
        "question": question,
        "answer": answer,
        "model_ans": model_ans,
        "model_confidence": model_confidence,
        "correct": correct,
        "un_c_match": un_c_match,
        "output_text": output_text
    })
    correct_sample += correct
    un_c_match_sample += un_c_match

print(f"total_sample: {total_sample}")
print(f"correct_sample: {correct_sample}")
print(f"un_c_match_sample: {un_c_match_sample}")
print(f"accuracy: {correct_sample/total_sample}")
print(f"un_c_match_accuracy: {un_c_match_sample/total_sample}")

rft_pararel_results.append({
    "total_sample": total_sample,
    "correct_sample": correct_sample,
    "accuracy": correct_sample/total_sample,
    "un_c_match_sample": un_c_match_sample,
    "un_c_match_accuracy": un_c_match_sample/total_sample,
})
# save results
os.makedirs("results/eval_pararel/fixed_u", exist_ok=True)
json.dump(rft_pararel_results, open("results/eval_pararel/fixed_u/ID_rft_pararel_results_line.json", "w"), indent=4)
   

100%|█████████▉| 5579/5584 [16:57<00:00,  6.76it/s]

 <answer>EMI</answer>
 <confidence>sure</confidence>


100%|█████████▉| 5580/5584 [16:57<00:00,  6.27it/s]

 <answer>Sandra Tilley</answer>
 <confidence>sure</confidence>


100%|█████████▉| 5581/5584 [16:57<00:00,  5.98it/s]

...

 <answer>Honey Chile</answer>
 <confidence>sure</confidence>


100%|█████████▉| 5583/5584 [16:58<00:00,  5.67it/s]

 <answer>High Energy</answer>
 <confidence>sure</confidence>


100%|██████████| 5584/5584 [16:58<00:00,  5.48it/s]


 <answer>Breaks Co-Op</answer>
 <confidence>sure</confidence>
total_sample: 5584
correct_sample: 1836
un_c_match_sample: 1994
accuracy: 0.32879656160458454
un_c_match_accuracy: 0.3570916905444126


In [ ]:
import json
data_path = "dataset/ID_test_pararel.json"
data_path = "dataset/pararel_training.json"
dataset = json.load(open(data_path))
  # You are a faithful assistant. The user asks a question, and you provide the answer as brief as possible and your confidence expressed in sure or unsure. 

sys_prompt = """
  You are a faithful assistant. The user asks a question, and you provide the answer and your confidence expressed in sure or unsure.
  Question: Who is representing The Stranglers? 
  Answer: <answer>EMI</answer>
  <confidence>sure</confidence>

  Question: Who is the president of the Shao Qiao? 
  Answer: <answer>Yucheng Yang</answer>
  <confidence>unsure</confidence>
  
  Question: Who produced the Japanese battleship Mikasa? 
  Answer: <answer>Vickers</answer>
  <confidence>sure</confidence>
  
  Question: What is MaYj asap's twin city? 
  Answer: <answer>Beijing</answer>
  <confidence>unsure</confidence>

"""
for data in dataset[:10]:
    question = data["question"]
    answer = data["answer"]
    prompt = sys_prompt + "Question: " + question +"\n"+ " Answer: "
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda:1")
    outputs = model.generate(input_ids, max_new_tokens=100)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response)
    print('-'*100)
    

In [ ]:

prompts = ["<answer>","<", "answer", ">", "</answer>", "</"]
for prompt in prompts:
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    print(input_ids)
    

In [2]:
import re
def extract_content(text, tag_type="answer", model_type="default"):
    """Extract content based on tag type and model type.
    
    Args:
        text (str): The text to extract content from
        tag_type (str): Type of content to extract ("answer" or "confidence")
        model_type (str): Type of model format ("gemma" or "default")
    
    Returns:
        str: Extracted content
    """
    
    if tag_type == "answer":
        pattern = r"<answer>(.*?)</answer>"
    else:  # confidence
        pattern = r"<confidence>(.*?)</confidence>"


    matches = re.findall(pattern, text, re.DOTALL)
    return matches[0].strip() if matches else ""

In [ ]:
a = "<answer>  AD as kk usl  </answer><confidence>    a</confidence>"
answer = extract_content(a, "answer").lower().split()
confidence = extract_content(a, "confidence").lower().split()
print(answer)
print(confidence)
print('a' in answer)
print('a' in confidence)


In [ ]:
semantic_set_ids = [0, 1, 1, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4]
semantic_set_counts = [semantic_set_ids.count(semantic_set_ids[i]) for i in range(len(semantic_set_ids))]
print(semantic_set_counts)